#### English to Swahili Translation Model

#### 1.0 Importing necessary libraries

In [1]:
# !pip install transformers
!pip install datasets
!pip install sentencepiece
!pip install matplotlib


You should consider upgrading via the 'C:\Users\peter\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\peter\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\peter\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [3]:
!pip install ipykernel

You should consider upgrading via the 'C:\Users\peter\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [4]:
import matplotlib.pyplot as plt
from datasets import load_dataset
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
import sentencepiece
import warnings
warnings.filterwarnings("ignore")

#### 1.1 loading the datasets

In [7]:
# Load the CSV dataset
df = load_dataset("csv", data_files=".dataset/ensw_v2.csv")

FileNotFoundError: Unable to find 'C:/Users/peter/Desktop/Kaps Hackathon/notebook\./dataset/ensw_v2.csv'

#### 1.2 Viewing the dataset

In [ ]:
print("Dataset object:\n\n", df)

: 

#### 1.3 Viewing the information about the dataset

In [ ]:
train_dataset = df['train']
print("Train dataset information:\n\n", train_dataset.features)

: 

#### 1.4  Viewing the split dataset information


In [ ]:
split_df = df['train'].train_test_split(train_size=0.9, seed=20)
print("\nSplit datasets:\n\n", split_df)

: 


#### 1.5 Loading the tokenizer and model

In [ ]:
model_checkpoint = "Helsinki-NLP/opus-mt-en-swc"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors="pt")
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

: 

#### 1.5 Set the maximum sequence length and define the preprocessing function

In [ ]:
max_length = 128

: 

In [ ]:
def preprocess_function(examples):
    inputs = str(examples['English sentence'])
    targets = str(examples['Swahili Translation'])

    model_inputs = tokenizer(
        inputs, text_target=targets, max_length=max_length, padding="max_length", truncation=True, return_tensors="pt"
    )
    model_inputs['decoder_input_ids'] = model_inputs['input_ids'].clone()

    return model_inputs

: 

#### 1.6 Preprocess the training and validation sets

In [ ]:
!pip install sentencepiece

: 

In [ ]:
# # loading the tokenizer and model
# model_checkpoint = "Helsinki-NLP/opus-mt-en-swc"
# tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors="pt")
# model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
# max_length = 128
# def preprocess_function(examples):
#     inputs = str(examples['English sentence'])
#     targets = str(examples['Swahili Translation'])

#     model_inputs = tokenizer(
#         inputs, text_target=targets, max_length=max_length, padding="max_length", truncation=True, return_tensors="pt"
#     )
#     model_inputs['decoder_input_ids'] = model_inputs['input_ids'].clone()

#     return model_inputs
train_dataset = split_df['train'].map(
    preprocess_function, batched=True, num_proc=4, remove_columns=["English sentence", "Swahili Translation"]
)


: 

In [ ]:
eval_dataset = split_df['test'].map(
    preprocess_function, batched=True, remove_columns=["English sentence", "Swahili Translation"]
)

: 

#### 1.7 Define the training arguments and create the trainer

In [ ]:

training_args = Seq2SeqTrainingArguments(
    output_dir='../model/',
    predict_with_generate=True,
    evaluation_strategy="steps",
    eval_steps=1000,
    save_steps=1000,
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=50,
    weight_decay=0.01,
    push_to_hub=False,
    logging_dir="../logs/",
    logging_steps=500,
)

: 

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

: 

#### 1.8 Train the model

In [ ]:
try:
    trainer.train()
except ValueError as e:
    print("Error during training:", e)

: 

#### 1.9 Evaluate the model on the validation set

In [ ]:
result = trainer.evaluate()
print(result)

: 

#### 2.0 Export the trained model

In [ ]:
# saving the model as a pkl  file
import pickle
pickle_out = open("model.pkl", mode = "wb")
pickle.dump(model, pickle_out)
pickle_out.close()

: 

In [ ]:
model.save_pretrained("../model/")
tokenizer.save_pretrained("../model/")

: 

#### 2.1 Creating a pipeline for translation


In [ ]:
translator = pipeline(
    "text2text-generation",
    model="../model/",
    tokenizer="../model/",
)

: 

#### 2.2 Prompt the user to enter a sentence for translation


In [ ]:
while True:
    text = input("Enter an English sentence for translation to Swahili (type 'exit' to quit): ")
    if text == "exit":
        break
    translated_text = translator(text, max_length=max_length, num_beams=5)[0]['generated_text']
    print(f"Translated text: {translated_text}")

: 

## THANK YOU ANY QUESTION

: 